In [7]:
from utils import DataFramePreprocessor
from utils import Morphology, SpaCyModel
from utils import VisualPOS, CustomTagger, Grammar

from pathlib import Path
import json

import numpy as np
import pandas as pd
import spacy
from spacy import displacy
from matplotlib import pyplot as plt

%config InlineBackend.figure_format = 'svg'

In [2]:
texts = pd.read_csv('examples_unique.csv', sep=',', encoding='utf-8')

with pd.option_context('display.max_colwidth', None):
    display(texts)

,id,uid,lang,version,text
0,1,01-zbeo71,EL,ελλΑΚ-1940,Κατὰ τὴν ἑρμηνείαν τῆς δηλώσεως βουλήσεως ἀναζητεῖται ἤ ἀληθὴς βούλησις ἄνευ προσηλώσεως εἱς τὰς λέξεις. (ΑΚ 173)
1,1,01-3nhd9b,EL,ελλΑΚ-2023,Κατά την ερμηνεία της δήλωσης βουλήσεως αναζητείται η αληθινή βούληση χωρίς προσήλωση στις λέξεις. (AK 173)
2,1,01-hdpkch,DE,γερΑΚ-2023,Bei der Auslegung einer Willenserklärung ist der wirkliche Wille zu erforschen und nicht an dem buchstäblichen Sinne des Ausdrucks zu haften. (BGB § 133)
3,1,01-mdawr5,DE,ελλΑΚ-1951,"Bei der Auslegung einer Willenserklärung wird der wirkliche Wille erforscht, ohne an den Worten zu haften."
4,2,02-6pjots,EL,ελλΑΚ-1940,"Διὰ τὰς προθεσμίας τὰς καθοριζομένας διὰ νόμου, δικαστικῆς αποφάσεως ἤ δικαιοπραξίας ἰσχύουσιν αἱ ἑρμηνευτικαὶ διατάξεις τῶν ἄρθρων 241 ἔως 246. (ΑΚ 240)"
5,2,02-ranxnx,EL,ελλΑΚ-2023,"Στις προθεσμίες που καθορίζονται με νόμο, δικαστική απόφαση ή δικαιοπραξία ισχύουν οι ερμηνευτικές διατάξεις των άρθρων 241 έως 246. (AK 240)"
6,2,02-e6b9hw,DE,γερΑΚ-2023,"Für die in Gesetzen, gerichtlichen Verfügungen und Rechtsgeschäften enthaltenen Frist- und Terminsbestimmung en gelten die Auslegungsvorschrif ten der §§ 187 bis 193. (BGB § 186)"
7,2,02-q6t2f5,DE,ελλΑΚ-1951,"Für die Fristen, welche durch Gesetz, gerichtliche Entscheidung oder Rechtsgeschäft bestimmt werden, gelten die Auslegungsvorschrif ten der Art. 241 bis 246."


In [3]:
preprocessed_df = DataFramePreprocessor(texts, remove_punctuation=False).preprocess()

with pd.option_context('display.max_colwidth', None):
    display(preprocessed_df.drop(columns=['text']))

,id,uid,lang,version,preprocessed
0,1,01-zbeo71,EL,ελλΑΚ-1940,Κατὰ τὴν ἑρμηνείαν τῆς δηλώσεως βουλήσεως ἀναζητεῖται ἤ ἀληθὴς βούλησις ἄνευ προσηλώσεως εἱς τὰς λέξεις.
1,1,01-3nhd9b,EL,ελλΑΚ-2023,Κατά την ερμηνεία της δήλωσης βουλήσεως αναζητείται η αληθινή βούληση χωρίς προσήλωση στις λέξεις.
2,1,01-hdpkch,DE,γερΑΚ-2023,Bei der Auslegung einer Willenserklärung ist der wirkliche Wille zu erforschen und nicht an dem buchstäblichen Sinne des Ausdrucks zu haften.
3,1,01-mdawr5,DE,ελλΑΚ-1951,"Bei der Auslegung einer Willenserklärung wird der wirkliche Wille erforscht, ohne an den Worten zu haften."
4,2,02-6pjots,EL,ελλΑΚ-1940,"Διὰ τὰς προθεσμίας τὰς καθοριζομένας διὰ νόμου, δικαστικῆς αποφάσεως ἤ δικαιοπραξίας ἰσχύουσιν αἱ ἑρμηνευτικαὶ διατάξεις τῶν ἄρθρων 000 ἔως 000."
5,2,02-ranxnx,EL,ελλΑΚ-2023,"Στις προθεσμίες που καθορίζονται με νόμο, δικαστική απόφαση ή δικαιοπραξία ισχύουν οι ερμηνευτικές διατάξεις των άρθρων 000 έως 000."
6,2,02-e6b9hw,DE,γερΑΚ-2023,"Für die in Gesetzen, gerichtlichen Verfügungen und Rechtsgeschäften enthaltenen Frist- und Terminsbestimmung en gelten die Auslegungsvorschrif ten der §§ 000 bis 000."
7,2,02-q6t2f5,DE,ελλΑΚ-1951,"Für die Fristen, welche durch Gesetz, gerichtliche Entscheidung oder Rechtsgeschäft bestimmt werden, gelten die Auslegungsvorschrif ten der Art. 000 bis 000."


In [4]:
el = SpaCyModel('el_core_news_lg')
de = SpaCyModel('de_dep_news_trf')

In [5]:
for i, text in enumerate(preprocessed_df['preprocessed']):
    lang = preprocessed_df['lang'][i]
    doc = el.get_doc(text) if lang == 'EL' else de.get_doc(text) 
    morph = [Morphology(token).get_morphology() for token in doc]
    print(json.dumps(morph, indent=4, ensure_ascii=False))
    # svg = displacy.render(doc, style='dep', jupyter=False)
    # filename = f'examples_unique_{i+1}.svg'
    # output_path = Path(f'img/{filename}')
    # output_path.open('w', encoding='utf-8').write(svg)
    # displacy.render(doc, style='dep', jupyter=True)

[
    {
        "Κατὰ": {}
    },
    {
        "τὴν": {
            "Case": "Acc",
            "Definite": "Def",
            "Gender": "Fem",
            "Number": "Sing",
            "PronType": "Art"
        }
    },
    {
        "ἑρμηνείαν": {
            "Case": "Acc",
            "Gender": "Fem",
            "Number": "Sing"
        }
    },
    {
        "τῆς": {
            "Case": "Gen",
            "Definite": "Def",
            "Gender": "Fem",
            "Number": "Sing",
            "PronType": "Art"
        }
    },
    {
        "δηλώσεως": {
            "Case": "Gen",
            "Gender": "Fem",
            "Number": "Sing"
        }
    },
    {
        "βουλήσεως": {
            "Case": "Gen",
            "Gender": "Fem",
            "Number": "Sing"
        }
    },
    {
        "ἀναζητεῖται": {
            "Aspect": "Imp",
            "Mood": "Ind",
            "Number": "Sing",
            "Person": "3",
            "Tense": "Pres",
            "VerbForm": "Fi

In [6]:
for i, text in enumerate(preprocessed_df['preprocessed']):
    lang = preprocessed_df['lang'][i]
    doc = el.get_doc(text) if lang == 'EL' else de.get_doc(text) 
    morph = [Morphology(token, grammar=True).get_morphology() for token in doc]
    print(json.dumps(morph, indent=4, ensure_ascii=False))

[
    {
        "Κατὰ": {
            "POS": "ADP"
        }
    },
    {
        "τὴν": {
            "POS": "DET",
            "Case": "Acc",
            "Definite": "Def",
            "Gender": "Fem",
            "Number": "Sing",
            "PronType": "Art"
        }
    },
    {
        "ἑρμηνείαν": {
            "POS": "NOUN",
            "Case": "Acc",
            "Gender": "Fem",
            "Number": "Sing"
        }
    },
    {
        "τῆς": {
            "POS": "DET",
            "Case": "Gen",
            "Definite": "Def",
            "Gender": "Fem",
            "Number": "Sing",
            "PronType": "Art"
        }
    },
    {
        "δηλώσεως": {
            "POS": "NOUN",
            "Case": "Gen",
            "Gender": "Fem",
            "Number": "Sing"
        }
    },
    {
        "βουλήσεως": {
            "POS": "NOUN",
            "Case": "Gen",
            "Gender": "Fem",
            "Number": "Sing"
        }
    },
    {
        "ἀναζητεῖται": {
  

In [16]:
custom_tagger = CustomTagger()


for i, text in enumerate(preprocessed_df['text'].to_list()):
    lang = preprocessed_df['lang'][i]
    doc = el.get_doc(text) if lang == 'EL' else de.get_doc(text)


TypeError: 'SpaCyModel' object is not callable